# Projeto 1 - Ciência dos Dados

Nome: João Victor de Almeida Braga

Nome: Gabriel de Sousa Fonseca

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import unicodedata 
import nltk

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
from sklearn.model_selection import train_test_split

nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\joaov\OneDrive\Documentos\Insper\2° semestre\CDados\Classificador_de_textos_automatico_P1_correto


Carregando a base de dados com os tweets classificados manualmente:

In [4]:
train = pd.read_excel('dados_treino_classificados.xlsx')
train.head()

,Mensagem,Target
0,Aversão para ler no Kindle está mais cara? Eu ...,CONT
1,"""Sapiens: uma breve história da humanidade rea...",CONT
2,"Paguei para embrulho de presente, e veio sem e...",PLAT
3,Eu esperava mais do livro ... Que final chato ...,CONT
4,O produto que chegou na minhas mãos veio com p...,ED


In [5]:
test = pd.read_excel('dados_teste_classificado.xlsx')
test.head()

,Mensagem,Target
0,"Tentei mas ,não deu. Não gosto de abandonar le...",CONT
1,Confesso que pelas avaliações eu esperava mais...,CONT
2,Particularmente não sou de livro de auto ajuda...,CONT
3,"Peço que verifiquem e reparem este erro, acred...",PLAT
4,"Não comprem, valor abusivo para e-book, amazon...",PLAT


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Targets:

$CONT$ - O target "CONT" é usado para classificar comentários que se referem a problemas que devem ser resolvidos pelo autor(relacionados ao conteúdo escrito), exemplo: "Focado apenas em quem deseja estudar melhor, textos rasos e superficiais, com dicas simples que conseguiríamos em algum post de blog na internet, não precisando necessariamente ter um livro para isso."

$PLAT$ - O target "PLAT" é usado para comentários de responsabilidade da Amazon(problemas de utilização da plataforma, valores e/ou problemas com kindle).

$ED$ - O target "ED" se refere a reclamações direcionadas a editora do livro(problemas de material, rasgos, amassos em diversos livros, erros de formatação, gráfia e organização do livro).

$OUTROS$ - O target "OUTROS" se refere a tudo aqui que não pode ser direcionado a nenhum dos 3 agentes citados anteriormente ou à ninguém(explicações do conteúdo do livro, criticas não construtivas e/ou extremamente curtas e/ou não objetivas, reclamações referente a transportadora).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [22]:
import re 

def cleanup(text):
    punctuation = '[´.,"!-:?;$''@#$%¨&*()_+=~><;/1234567890]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


def separa_palavras(series):
    coment_juntos = ''
    for i in series:
        coment_juntos += i + ''

    coment_juntos = coment_juntos.lower()
    
    return coment_juntos

def limpa_acentos(str):
    sem_acentos = unicodedata.normalize("NFD", str)
    sem_acentos = sem_acentos.encode("ascii", "ignore")
    sem_acentos = sem_acentos.decode("utf-8")

    return sem_acentos


def stop_words(texto): 
    palavras_limpas = []
    texto_separado = word_tokenize(texto, language='portuguese')

    stopwords_pt = set(stopwords.words('portuguese'))
    
    for x in texto_separado:
        if x not in stopwords_pt:
            palavras_limpas.append(x)
    
    return palavras_limpas

def simplifica_palavra(list_texto):
    list_texto_simples = []
    stemmer = SnowballStemmer("portuguese")
    for palavra in list_texto:
        list_texto_simples.append(stemmer.stem(palavra))

    return list_texto_simples


simplifica_palavra(stop_words(limpa_acentos(cleanup(separa_palavras(test.Mensagem)))))

['compr',
 'livr',
 'intuit',
 'aprend',
 'sobr',
 'culin',
 'por',
 'livr',
 'inutil',
 'nao',
 'prest',
 'nao',
 'ajud',
 'nad',
 'tod',
 'receit',
 'sao',
 'sup',
 'dific',
 'diz',
 'pratic',
 'rap',
 'simpl']

In [20]:
def trata_df(df,target):
    df_target = df.loc[df.Target == target]
    lista_df_limpo = simplifica_palavra(stop_words(limpa_acentos(cleanup(separa_palavras(df_target.Mensagem)))))
    abs_df_target = pd.Series(lista_df_limpo).value_counts()
    rel_df_target = pd.Series(lista_df_limpo).value_counts(True)
    return [abs_df_target,rel_df_target, lista_df_limpo]


trata_df(train,'CONT')[2]

['aversa',
 'ler',
 'kindl',
 'car',
 'invest',
 'ness',
 'aparelh',
 'achand',
 'livr',
 'digit',
 'sair',
 'cont',
 'brasileir',
 'le',
 'pouc',
 'conseg',
 'usar',
 'part',
 'dinheir',
 'ganh',
 'intuit',
 'adquir',
 'pouc',
 'conhec',
 'acab',
 'tend',
 'tip',
 'decepca',
 'respondamm',
 'justif',
 'prec',
 'ta',
 'saind',
 'car',
 'papel',
 'pixels',
 'tel',
 'atualizaca',
 'ontem',
 'dia',
 'setembr',
 'prec',
 'dess',
 'livr',
 'quas',
 'r',
 'hoj',
 'dia',
 'comentari',
 'acim',
 'r',
 'prec',
 'versa',
 'impress',
 'continu',
 'injustific',
 'valor',
 'esper',
 'respost',
 'porqu',
 'seri',
 'cab',
 'denunc',
 'obrigadosapiens',
 'brev',
 'histor',
 'human',
 'realment',
 'realment',
 'livr',
 'interess',
 'ja',
 'hom',
 'deus',
 'decepcion',
 'pois',
 'pouc',
 'nad',
 'fal',
 'cheg',
 'amanha',
 'esper',
 'livr',
 'final',
 'chat',
 'gent',
 'cad',
 'filh',
 'casal',
 'decepcion',
 'autor',
 'chato',
 'esper',
 'livr',
 'esper',
 'lig',
 'titul',
 'algum',
 'ide',
 'apresent'

In [27]:
cont_relativo = trata_df(train,'CONT')[1]
cont_absoluto = trata_df(train, 'CONT')[0]
palavras_cont = trata_df(train, 'CONT')[2]

ed_relativo = trata_df(train,'ED')[1]
ed_absoluto = trata_df(train, 'ED')[0]
palavras_ed = trata_df(train, 'ED')[2]

plat_relativo = trata_df(train,'PLAT')[1]
plat_absoluto = trata_df(train, 'PLAT')[0]
palavras_plat = trata_df(train, 'PLAT')[2]

outros_relativo = trata_df(train,'OUTROS')[1]
outros_absoluto = trata_df(train, 'OUTROS')[0]
palavras_outros = trata_df(train, 'OUTROS')[2]

406

In [9]:
def total_possibilidade(total_palavras):
    sem_repetição = []
    for x in total_palavras:
        if x not in sem_repetição:
            sem_repetição.append(x)

    return sem_repetição

palavras_geral = palavras_cont + palavras_ed + palavras_plat + palavras_outros
total_de_possibilidades = total_possibilidade(palavras_geral)

In [13]:
def Prob_target(df_target ,todas_as_palavras ):
    prob = sum(df_target) / len(todas_as_palavras)

    return prob

P_cont = Prob_target(cont_absoluto, palavras_geral)

P_ed = Prob_target(ed_absoluto, palavras_geral)

P_plat = Prob_target(plat_absoluto, palavras_geral)

P_outros = Prob_target(outros_absoluto, palavras_geral)



In [28]:
#João tá fazendo coisa aqui
def P_frase_cond_suavizada(comentário,df, alpha,total_de_possibilidades):
    p_frase_dado_target = 1
    for palavra in comentário:
        p_frase_dado_target *= (df[palavra] + alpha) / (sum(df) + alpha * total_de_possibilidades)

    return p_frase_dado_target


def define_target(Posterior1,Posterior2,Posterior3):

    lista_probs_posteriores = [Posterior1, Posterior2, Posterior3]

    if max(lista_probs_posteriores) == lista_probs_posteriores[0]:
        return 'CONT'
    
    elif max(lista_probs_posteriores) == lista_probs_posteriores[1]:

        return 'PLAT'

    elif max(lista_probs_posteriores) == lista_probs_posteriores[2]:

        return 'ED'

    elif max(lista_probs_posteriores) == lista_probs_posteriores[3]:

        return 'OUTROS'
        

def Classificador(df):
    abs_cont = trata_df(df, 'CONT')

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**